# Payoff Matrices (part 2)

> This module contains payoff matrices for different evolutionary games
>
> Part 2 contains payoff matrices for the following games
> - Encanacao et al. 2016
> - Vasconcelos et al. 2014

In [10]:
#| default_exp `dummy_exp`

In [11]:
#| hide
# We could like to export all payoff matrices to the same module, `payoffs`.
# However,  we must set a default_exp and if we use the same module as
# another notebook, we will overwrite all existing exports with those in
# this file. Hence we set a dummy default_exp, `dummy_exp`, and each time
# we export a cell, we specify the `payoffs` module.

In [12]:
#| hide
#| export
from nbdev.showdoc import *
from fastcore.test import test_eq, test_close
from gh_pages_example.utils import *
from gh_pages_example.types import *
import typing

import fastcore.test
import numpy as np
import nptyping

In [13]:
np.set_printoptions(suppress=True) # don't use scientific notation

In [14]:
#| export payoffs
def payoffs_encanacao_2016(models):
    names = ['b_r', 'b_s', 'c_s', 'c_t', 'σ']
    b_r, b_s, c_s, c_t, σ = [models[k] for k in names]
    payoffs = {}
    n_players = 3
    n_sectors = 3
    n_strategies_per_sector = [2, 2, 2]
    n_strategies_total = 6
    index_min = "0-0-0" # All players are from the first sector, playing that sector's first strategy
    index_max = "5-5-5" # All players are from the third sector, playing that sector's second strategy
    # Note: The seperator makes it easy to represent games where n_strategies_total >= 10.
    
    # It is also trivial to define a vector which maps these indexes to strategy profiles
    # As sector order is fixed we could neglect to mention suscripts for each sector
    strategy_names = ["D", "C", "D", "C", "D", "C"]
    
    zero = np.zeros(b_r.shape[0])
    # As in the main text
    payoffs["C-C-C"] = {"P3": b_r-2*c_s,
                        "P2": σ+b_s-c_t,
                        "P1": σ+b_s}
    payoffs["C-C-D"] = {"P3": -c_s,
                        "P2": b_s-c_t,
                        "P1": zero}
    payoffs["C-D-C"] = {"P3": b_r-c_s,
                        "P2": zero,
                        "P1": b_s}
    payoffs["C-D-D"] = {"P3": zero,
                        "P2": σ,
                        "P1": σ}
    payoffs["D-C-C"] = {"P3": zero,
                        "P2": σ-c_t,
                        "P1": σ}
    payoffs["D-C-D"] = {"P3": zero,
                        "P2": -c_t,
                        "P1": zero}
    payoffs["D-D-C"] = {"P3": zero,
                        "P2": zero,
                        "P1": zero}
    payoffs["D-D-D"] = {"P3": zero,
                        "P2": σ,
                        "P1": σ}
    
    # The following indexes capture all strategy profiles where each player is fixed to a unique sector
    # (and player order does not matter, so we need only consider one ordering of sectors).
    payoffs["4-2-0"] = payoffs["D-D-D"]
    payoffs["4-2-1"] = payoffs["D-D-C"]
    payoffs["4-3-0"] = payoffs["D-C-D"]
    payoffs["4-3-1"] = payoffs["D-C-C"]
    payoffs["5-2-0"] = payoffs["C-D-D"]
    payoffs["5-2-1"] = payoffs["C-D-C"]
    payoffs["5-3-0"] = payoffs["C-C-D"]
    payoffs["5-3-1"] = payoffs["C-C-C"]
    return {**models, "payoffs": payoffs}

## Vasconselos et al. 2014

They introduce a model of a Collective Risk Dilemma. It is a variant of the
public goods game where players must achieve a target level of contributions
to avoid risking a disaster which destroys the group's endowments.

We compute payoffs when players contribute $0$ or a fixed $c$ proportion of
their endowment as a contribution in
a game with up to $n$ participants. To do this, we compute the payoffs as a
function of the number of contributors, then use that function for each
relevant strategy profile.

In [15]:
@multi
def build_payoffs(models:dict):
    return models.get('payoffs_key')

@method(build_payoffs, 'vasconcelos_2014_primitives')
def build_payoffs(models:dict):
    names = ['payoffs_state', 'c', 'T', 'b_r', 'b_p', 'r']
    payoffs_state, c, T, b_r, b_p, r = [models[k] for k in names]
    strategy_counts = payoffs_state['strategy_counts']
    n_r = strategy_counts["2"]
    n_p = strategy_counts["4"]
    risk = r * (n_r * c * b_r + n_p * c * b_p < T)
    # The payoffs must be computed for each strategy type in the interaction.
    # In games where we employ hypergeometric sampling, we usually do not
    # care about player order in the interaction. If order did matter, then
    # we would represent the payoffs per strategy still but it would capture
    # the expected payoffs given how likely a player of that strategy was to
    # play in each node of the extensive-form game. Non-players of type 0
    # usually do not have payoffs.
    payoffs = {"1": (1 - risk) * b_r,  # rich_free_rider
               "2": (1 - risk) * c * b_r,  # rich_contributor
               "3": (1 - risk) * b_p,  # poor_free_rider
               "4": (1 - risk) * c * b_p}  # poor_contributor
    return {**models, "payoff_primitives": payoffs}

@method(build_payoffs, 'vasconcelos_2014')
def build_payoffs(models:dict):
    profiles = create_all_profiles({'n_players': models.get('n_players', 5),
                                    'n_strategies': [2, 2]})['profiles']
    payoffs = {}
    for profile in profiles:
        profile_tuple = thread_macro(profile,
                                    (str.split, "-"),
                                    (map, int, "self"),
                                    list,
                                    reversed,
                                    list,
                                    np.array,
                                    )
        strategy_counts = {f"{i}": np.sum(profile_tuple == i) for i in range(5)}
        payoffs_state = {'strategy_counts': strategy_counts}
        primitives = thread_macro(models,
                                  (assoc,
                                  'payoffs_state', payoffs_state,
                                  'payoffs_key', "vasconcelos_2014_primitives"),
                                  build_payoffs,
                                  (get, "payoff_primitives"),
                                  )
        payoffs[profile] = {}
        for i, strategy in enumerate(profile_tuple):
            if strategy == 0:
                continue
            elif strategy == 1:
                payoffs[profile][f"P{i+1}"] = primitives['1']
            elif strategy == 2:
                payoffs[profile][f"P{i+1}"] = primitives['2']
            elif strategy == 3:
                payoffs[profile][f"P{i+1}"] = primitives['3']
            elif strategy == 4:
                payoffs[profile][f"P{i+1}"] = primitives['4']
            else:
                continue
    return {**models, "payoffs": payoffs}

Here are a few simple tests of the payoff primitives for their model.

In [16]:
models = {'payoffs_state': {'strategy_counts': {"2": 2,
                                                "4": 4}},
          'c': 0.5,
          'T': 2,
          'b_r': 4,
          'b_p': 2,
          'r': 0.5,
          'payoffs_key': 'vasconcelos_2014_primitives'}
models = build_payoffs(models)
fastcore.test.test_eq(models['payoff_primitives'],
                      {'1': 4,
                       '2': 2,
                       '3': 2,
                       '4': 1})
models = {**models, 
          'payoffs_state': {'strategy_counts': {"2": 0,
                                                "4": 1}},}
models = build_payoffs(models)
fastcore.test.test_eq(models['payoff_primitives'],
                      {'1': 2,
                       '2': 1,
                       '3': 1,
                       '4': 0.5})

We quickly check that we can generate payoffs for each of the 5**5 possible
interactions in their model.

In [17]:
models = {'c': 0.5,
          'T': 2,
          'b_r': 4,
          'b_p': 2,
          'r': 0.5,
          'payoffs_key': 'vasconcelos_2014'}
models = build_payoffs(models)
fastcore.test.test_eq(len(models['payoffs']), 5**5)

If we are unwilling to use the 5**5 possible strategy profiles for computing
the transition matrices for the evolutionary system, we can always restrict
our attention to the payoffs given the number of contributors from each sector.
We often use hypergeometric sampling anyways when computing the success of
each strategy in the evolutionary system.

Note: we have yet to create a general method for hypergeometric sampling when
computing the success of an arbitrary model.

In [18]:
#| hide
import nbdev; nbdev.nbdev_export()